<a href="https://colab.research.google.com/github/compi1234/pyspch/blob/master/test/audio_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Audio Test Notebook 

especially the JAVASCRIPT pieces in audio

Date: 17/05/2021

In [6]:
%matplotlib inline
#from IPython import notebook
from IPython.display import display, clear_output, Audio, HTML, Javascript
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

import librosa
try:
  import google.colab
  IN_COLAB = True 
  ! pip install git+https://github.com/compi1234/pyspch.git
except:
  IN_COLAB = False

# verify the IPython version
import IPython
if IPython.version_info[0] >= 6:
    Audio_args = {'normalize':False}
else:
    print("Warning: you are using IPython<6 \n IPython.display.Audio() will automatically normalize audio output")
    Audio_args = {}
import pyspch.audio as audio
import pyspch.spectrogram as specg
import pyspch.utils as spch_utils
import pyspch.display as spch_disp
#import pyspch.display_ly as spch_disp
import pyspch

SAMPLE_WAV_URL = 'https://homes.esat.kuleuven.be/~spchlab/data/misc/train.wav'
SAMPLE_WAV_13dB_SNGER = 'https://homes.esat.kuleuven.be/~spchlab/data/13dB/Psnger22.wav'

## 1. Read from file

In [ ]:
wavname = SAMPLE_WAV_URL
wavdata,sample_rate= audio.load(wavname)
# 
print(wavdata.shape, sample_rate)
fig = plt.figure(figsize=(6,2))
plt.plot(wavdata)
# if you want the plot above the audio widget, add line below
# plt.show()
display(Audio(data=wavdata,rate=sample_rate,autoplay=False,**Audio_args))

In [ ]:
audio.play(wavdata,sample_rate=sample_rate,wait=True,io_device='js')

In [ ]:
# Getting values out of GUI via globals

In [ ]:
global global_value
def box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
class demo(widgets.VBox):
    def __init__(self,value=10):
        self.value = value
        super().__init__()
        self.wg_button = widgets.Button(description='Click',layout=box_layout())
        self.wg_button.on_click(self.click_button)
        self.out = widgets.Output(layout=box_layout())
        self.children = [self.wg_button, self.out]

        
    def click_button(self,b): 
        global global_value
        self.value = 2. * self.value
        global_value = self.value
        with self.out:
            print("New value: ",self.value)
            
demo(value=3)

In [ ]:
global_value

## Interfacing HTML and JS 
- get the HTML (GUI) in place
- define the javascript code after finishing all HTML, 
    + do it all in the same cell !

In [20]:
%%html
<input id='x' type='number' value=2>
x
<input id='y' type='number' value=-1> 
<input type="button" value="=" onclick="DoIt('x','y','z')">
<b id='z'></b>
<script>
var z
function Multi(x,y) {
    console.log("Multi - original")
    return Number(x) * Number(y)
    }
function DoIt(xid,yid,zid) {
    x = document.getElementById(xid).value
    y = document.getElementById(yid).value
    z = Multi(x,y)
    document.getElementById(zid).innerHTML = z
    console.log("z= ",x,y,z)
}
</script>

You can split definition of HTML, Javascript and execution
Just be careful not to run any Javascript code before the HTML elements it requires are defined

In [9]:
_HTML1_ = """
<input id='x1' type='number' value=2>
x
<input id='y1' type='number' value=-1> 
<input type='button' value='=' onclick='DoIt1("x1","y1","z1")'>
<b id='z1'></b>
"""

_JS1_ = """
function Multi1(x,y) {
    console.log('Multi1')
    return Number(x) * Number(y)
    }
function DoIt1(xid,yid,zid) {
    console.log('DoIt1')    
    x = document.getElementById(xid).value
    y = document.getElementById(yid).value
    var z = Multi1(x,y)
    document.getElementById(zid).innerHTML = z
    console.log('z= ',x,y,z)
}
"""

display(HTML(_HTML1_ + "<script>" + _JS1_  + "</script>"))


In [11]:
python_variable = 3.5
_JS2_ = """
console.log("Executing JS2")
console.log("old= ",old)
var my_js_variable = 2 * old
console.log("new= ",my_js_variable)
"""
txt2 = "var old=%f\n " % python_variable
Javascript(txt2+_JS2_)

<IPython.core.display.Javascript object>

In [26]:
%%html
<b id='x3'>3</b> x 
<b id='y3'>7</b> = 
<b id='z3'>0</b>

In [28]:
# passing arguments to javascript functions
# definition + stacking several functions

In [29]:
_JS3_ = """
//var x = document.getElementById('x3').innerHTML;
//var y = document.getElementById('y3').innerHTML;
document.getElementById('z3').innerHTML = Number(x) *Number(y)
console.log("Executing _JS3_")
function Multi3() {
    console.log("Multi3")
    var x = document.getElementById('x3').innerHTML;
    var y = document.getElementById('y3').innerHTML;
    document.getElementById('z3').innerHTML = Number(x) *Number(y)
    console.log("new= ")
    }
function Update3(x,y) {
    document.getElementById('x3').innerHTML=x
    document.getElementById('y3').innerHTML=y
    }
Multi3()
//console.log("new= ",my_js_variable)
"""

#Javascript(_JS3_)
Javascript(_JS3_ + "Update3(225,3.2) + Multi3() ")

<IPython.core.display.Javascript object>

## Javascript Audio

In [ ]:
# Ipython.notebook.kernel is a mechanism to get things back from javascript to python
# but .....   works in standard notebook, not in lab

In [ ]:
from IPython.display import HTML

input_form = """
<div style="background-color:gainsboro; border:solid black; width:300px; padding:20px;">
Variable Name: <input type="text" id="var_name" value="foo"><br>
Variable Value: <input type="text" id="var_value" value="bar"><br>
<button onclick="set_value()">Set Value</button>
</div>
"""

javascript = """
<script type="text/Javascript">
    function set_value(){
        var var_name = document.getElementById('var_name').value;
        var var_value = document.getElementById('var_value').value;
        var command = var_name + " = '" + var_value + "'";
        console.log("Executing Command: " + command);
        
        var kernel = IPython.notebook.kernel;
        kernel.execute(command);
    }
</script>
"""

HTML(input_form + javascript)

In [ ]:
print(foo)

In [ ]:
%%html
<h2>Play Audio File: from file or url</h2>
<p>
<label for="file">Select a File : </label>
<input id="audio_file" type="file" accept="audio/*" />
</p>
<p>
<label for="url">Complete the URL : </label>
<input id="audio_url" type="url" name="url" 
       value="https://homes.esat.kuleuven.be/~spchlab/data/misc/"
       pattern="https://.*" size="60"
       required>
</p>
<p>
<audio controls id="audio_player"></audio>
</p>

<script>

audio_file.onchange = function(){
    var files = this.files;
    var file = URL.createObjectURL(files[0]); 
    audio_player.src = file; 
};

audio_url.onchange = function(){
    var url = this.value; 
    audio_player.src = url; 
};

</script>

In [ ]:
display.JavaScript("""
const time = 2000
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: {channelCount: 2} })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
""")


In [ ]:
%%javascript
var variable2 = "this_could_be_any_javascript_string";
// command is a string containing Python code
var command = "fromJavaScript='" + variable2 + "'";
IPython.notebook.kernel.execute(command);

In [ ]:
print(fromJavaScript)

In [ ]:
%%html
<button id="recordButton">Record</button>
<button id="stopButton">Stop</button>


In [7]:
%%html
<button id="recordButton">Record</button>
<button id="stopButton" disabled>Stop</button>
<button id="pauseButton" disabled>Pause</button>
<script>
//webkitURL is deprecated but nevertheless
URL = window.URL || window.webkitURL;

var gumStream; 						//stream from getUserMedia()
var recorder; 						//MediaRecorder object
var chunks = [];					//Array of chunks of audio data from the browser
var extension;

var recordButton = document.getElementById("recordButton");
var stopButton = document.getElementById("stopButton");
var pauseButton = document.getElementById("pauseButton");

//add events to those 2 buttons
recordButton.addEventListener("click", startRecording);
stopButton.addEventListener("click", stopRecording);
pauseButton.addEventListener("click", pauseRecording);

// true on chrome, false on firefox
console.log("audio/webm:"+MediaRecorder.isTypeSupported('audio/webm;codecs=opus'));
// false on chrome, true on firefox
console.log("audio/ogg:"+MediaRecorder.isTypeSupported('audio/ogg;codecs=opus'));

if (MediaRecorder.isTypeSupported('audio/webm;codecs=opus')){
	extension="webm";
}else{
	extension="ogg"
}


function startRecording() {
	console.log("recordButton clicked");

	/*
		Simple constraints object, for more advanced audio features see
		https://addpipe.com/blog/audio-constraints-getusermedia/
	*/
    
    var constraints = {audio: true}

 	/*
    	Disable the record button until we get a success or fail from getUserMedia() 
	*/

	recordButton.disabled = true;
	stopButton.disabled = false;
	pauseButton.disabled = false

	/*
    	We're using the standard promise based getUserMedia() 
    	https://developer.mozilla.org/en-US/docs/Web/API/MediaDevices/getUserMedia
	*/

	navigator.mediaDevices.getUserMedia(constraints).then(function(stream) {
		console.log("getUserMedia() success, stream created, initializing MediaRecorder");

		/*  assign to gumStream for later use  */
		gumStream = stream;

		var options = {
	      audioBitsPerSecond :  256000,
	      videoBitsPerSecond : 2500000,
	      bitsPerSecond:       2628000,
	      mimeType : 'audio/'+extension+';codecs=opus'
	    }

	    //update the format 
		document.getElementById("formats").innerHTML='Sample rate: 48kHz, MIME: audio/'+extension+';codecs=opus';

		/* 
			Create the MediaRecorder object
		*/
		recorder = new MediaRecorder(stream, options);

		//when data becomes available add it to our attay of audio data
	    recorder.ondataavailable = function(e){
	    	console.log("recorder.ondataavailable:" + e.data);
	    	
	    	console.log ("recorder.audioBitsPerSecond:"+recorder.audioBitsPerSecond)
	    	console.log ("recorder.videoBitsPerSecond:"+recorder.videoBitsPerSecond)
	    	console.log ("recorder.bitsPerSecond:"+recorder.bitsPerSecond)
	      	// add stream data to chunks
	      	chunks.push(e.data);
	      	// if recorder is 'inactive' then recording has finished
	      	if (recorder.state == 'inactive') {
	          // convert stream data chunks to a 'webm' audio format as a blob
	          const blob = new Blob(chunks, { type: 'audio/'+extension, bitsPerSecond:128000});
	          createDownloadLink(blob)
	      	}
	    };

	    recorder.onerror = function(e){
	    	console.log(e.error);
	    }

	    //start recording using 1 second chunks
	    //Chrome and Firefox will record one long chunk if you do not specify the chunck length
	    recorder.start(1000);

    	//recorder.start();
    }).catch(function(err) {
	  	//enable the record button if getUserMedia() fails
    	recordButton.disabled = false;
    	stopButton.disabled = true;
    	pauseButton.disabled = true
	});
}

function pauseRecording(){
	console.log("pauseButton clicked recorder.state=",recorder.state );
	if (recorder.state=="recording"){
		//pause
		recorder.pause();
		pauseButton.innerHTML="Resume";
	}else if (recorder.state=="paused"){
		//resume
		recorder.resume();
		pauseButton.innerHTML="Pause";

	}
}

function stopRecording() {
	console.log("stopButton clicked");

	//disable the stop button, enable the record too allow for new recordings
	stopButton.disabled = true;
	recordButton.disabled = false;
	pauseButton.disabled = true;

	//reset button just in case the recording is stopped while paused
	pauseButton.innerHTML="Pause";
	
	//tell the recorder to stop the recording
	recorder.stop();

	//stop microphone access
	gumStream.getAudioTracks()[0].stop();
}

function createDownloadLink(blob) {
	
	var url = URL.createObjectURL(blob);
	var au = document.createElement('audio');
	var li = document.createElement('li');
	var link = document.createElement('a');

	//add controls to the <audio> element
	au.controls = true;
	au.src = url;

	//link the a element to the blob
	link.href = url;
	link.download = new Date().toISOString() + '.'+extension;
	link.innerHTML = link.download;

	//add the new audio and a elements to the li element
	li.appendChild(au);
	li.appendChild(link);

	//add the li element to the ordered list
	recordingsList.appendChild(li);
}
</script>

In [ ]:
from ipywebrtc import AudioStream, AudioRecorder
import ipywidgets as widgets

In [ ]:
audio_stream = AudioStream.from_url(SAMPLE_WAV_URL)

In [ ]:
display(audio_stream)

In [ ]:
audio_stream.playing=False

In [ ]:
play_button = widgets.ToggleButton(description="Play")
widgets.jslink((play_button, 'value'), (audio_stream, 'playing'))
widgets.VBox(children=[audio_stream, play_button])

In [ ]:
audio_stream

In [ ]:
from unittest import TestCase

from aiortc.codecs import get_decoder, get_encoder
from aiortc.rtcrtpparameters import RTCRtpCodecParameters

BOGUS_CODEC = RTCRtpCodecParameters(
    mimeType="audio/bogus", clockRate=8000, channels=1, payloadType=0
)


class CodecsTest(TestCase):
    def test_get_decoder(self):
        with self.assertRaises(ValueError):
            get_decoder(BOGUS_CODEC)

    def test_get_encoder(self):
        with self.assertRaises(ValueError):
            get_encoder(BOGUS_CODEC)

In [ ]:
from aiortc.contrib.media import MediaPlayer
player=MediaPlayer(SAMPLE_WAV_URL)

In [ ]:
display(player)

In [ ]:
help(player.audio)

In [ ]:
player.audio.recv()

In [ ]:
import asyncio
loop = asyncio.get_event_loop()

loop.run_until_complete(
    run(pc=pc, player=player)
)

In [8]:
%%html
<input type="file" accept="audio/*" capture id="recorder">
<audio id="player" controls></audio>
<script>
  const recorder = document.getElementById('recorder');
  const player = document.getElementById('player');

  recorder.addEventListener('change', function(e) {
    const file = e.target.files[0];
    const url = URL.createObjectURL(file);
    // Do something with the audio file.
    player.src = url;
  });
</script>

In [ ]:
%%html
<audio id="player" controls></audio>
<script>
  const player = document.getElementById('player');

  const handleSuccess = function(stream) {
    const context = new AudioContext();
    const source = context.createMediaStreamSource(stream);
    const processor = context.createScriptProcessor(1024, 1, 1);

    source.connect(processor);
    processor.connect(context.destination);

    processor.onaudioprocess = function(e) {
      // Do something with the data, e.g. convert it to WAV
      console.log(e.inputBuffer);
    };
  };

  navigator.mediaDevices.getUserMedia({ audio: true, video: false })
      .then(handleSuccess);
</script>

In [ ]:
%%html
<audio id="player" controls></audio>
<script>
  const player = document.getElementById('player');

  const handleSuccess = function(stream) {
    if (window.URL) {
      player.srcObject = stream;
    } else {
      player.src = stream;
    }
  };

  navigator.mediaDevices.getUserMedia({ audio: true, video: false })
      .then(handleSuccess);
</script>

In [ ]:
%%html
<a id="download">Download</a>
<button id="stop">Stop</button>
<script>
  let shouldStop = false;
  let stopped = false;
  const downloadLink = document.getElementById('download');
  const stopButton = document.getElementById('stop');

  stopButton.addEventListener('click', function() {
    shouldStop = true;
  });

  const handleSuccess = function(stream) {
    const options = {mimeType: 'audio/webm'};
    const recordedChunks = [];
    const mediaRecorder = new MediaRecorder(stream, options);

    mediaRecorder.addEventListener('dataavailable', function(e) {
      if (e.data.size > 0) {
        recordedChunks.push(e.data);
      }

      if(shouldStop === true && stopped === false) {
        mediaRecorder.stop();
        stopped = true;
      }
    });

    mediaRecorder.addEventListener('stop', function() {
      downloadLink.href = URL.createObjectURL(new Blob(recordedChunks));
      downloadLink.download = 'acetest.wav';
    });

    mediaRecorder.start();
  };